In [1]:
import torch
import torchvision

In [2]:
n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [3]:
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

0it [00:00, ?it/s]

9920512it [00:03, 3114696.14it/s]                             


Extracting /files/MNIST/raw/train-images-idx3-ubyte.gz


32768it [00:00, 6944167.01it/s]
0it [00:00, ?it/s]

Extracting /files/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 3628116.74it/s]                             
8192it [00:00, 2131761.90it/s]


Extracting /files/MNIST/raw/t10k-images-idx3-ubyte.gz
Extracting /files/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [4]:
examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)

In [5]:
example_data.shape

torch.Size([1000, 1, 28, 28])

In [6]:
import matplotlib.pyplot as plt

fig = plt.figure()
for i in range(6):
  plt.subplot(2,3,i+1)
  plt.tight_layout()
  plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
  plt.title("Ground Truth: {}".format(example_targets[i]))
  plt.xticks([])
  plt.yticks([])
fig

<Figure size 640x480 with 6 Axes>

In [7]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [9]:
network = Net()
optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                      momentum=momentum)

In [10]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [16]:
def train(epoch):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = network(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
      torch.save(network.state_dict(), './results/model.pth')
      torch.save(optimizer.state_dict(), './results/optimizer.pth')

In [17]:
def test():
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = network(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [18]:
# test()
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.273459
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.255804
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.253477
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.276307
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.205931
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.142368
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.222990
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.045199
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.989158
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.872446
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.830397
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.789129
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.745549
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.592391
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.423177
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.429330
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.316790
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.193643
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.393684
Train Epoch: 1 [12160/60000 (20%)]	Loss: 1.174951
Train Epoch: 1 [12800/60000 (

/opt/conda/lib/python3.7/site-packages/torch/nn/_reduction.py:46: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 0.2059, Accuracy: 9404/10000 (94%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.616770
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.371861
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.550599
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.427689
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.389997
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.491877
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.754018
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.455917
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.271281
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.516181
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.584363
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.484237
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.386958
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.537496
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.549177
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.195472
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.381628
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.151023
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.362783
Train Epoch: 2 [12160